##SCRIPT TO CHECK THE DURATION/TIMING OF TRIALS IN MEG USING THE PHOTODIODE

In [7]:
import mne
import pandas as pd
import numpy as np
import glob
import pylab
import pandas as pd
import pickle
import matplotlib.pyplot as plt

from hv_proc.utilities import mrk_template_writer

S = 'test360'
proj_path = "/misc/data12/sjapee/Sebastian-OrientationImagery/Data/"
raw_dir = proj_path + '20240629/' + S + '/'
logfile_dir = proj_path + 'sheets/'
run_Length = 1
stim_duration = 3

In [8]:
#New way of defining onsets not based on standard deviation but based on finding photodiode after the trigger
def defineOnsets2(raw, events):
    log_statements = []
    pd_dat = raw[raw.ch_names.index('UADC016-2104')][0][0]
    if np.mean(np.abs(pd_dat))>0.6: # this should be fine but may not be good if there are a looot of zeros
        # zero-centering & making everything positive
        pd_dat = np.abs(pd_dat-np.mean(pd_dat[0:100]))
        # scaling channel to 0-1
        pd_dat = (pd_dat-np.min(pd_dat)) / (np.max(pd_dat)-np.min(pd_dat))

        # using the event, we are now looking in the next 50ms of the photodiode channel (digital) when the signal goes above 0.5
        t_samples = int(0.05/(1/raw.info['sfreq']))
        threshold =  .5

        # when we found the events we calculate the trigger delay. 
        pd_events = [events[i,0]+np.where(pd_dat[events[i,0]:events[i,0]+t_samples]<threshold)[0][0] for i in range(len(events))]
        pd_onsets = []
        pd_offsets = []

        for event in events:
            # Find onset
            onset_index = event[0] + np.argmax(pd_dat[event[0]:event[0] + t_samples] > threshold)
            pd_onsets.append(onset_index)

            # Find offset
            pd_offset_index = onset_index + np.argmax(pd_dat[onset_index:] < threshold)
            pd_offsets.append(pd_offset_index)
        trigger_delay = 1000. * (pd_events-events[:, 0]) / raw.info['sfreq']
        # sometimes this channel records ON as UP and sometimes as DOWN. This if-loop makes sure we always find the onsets
        if np.mean(trigger_delay)==0:
            pd_events = [events[i,0]+np.where(pd_dat[events[i,0]:events[i,0]+t_samples]>threshold)[0][0] for i in range(len(events))]
            trigger_delay = 1000. * (pd_events-events[:, 0]) / raw.info['sfreq']
        log_statements.append(('Trigger delay removed (μ ± σ): %0.1f ± %0.1f ms')
            % (np.mean(trigger_delay), np.std(trigger_delay)))
        if np.mean(trigger_delay)>10:
            events[:,0] = events[:,0]+5
            log_statements.append(f'photo diode and parallel port triggers are too far apart. using parallel port trigger ')
        else:
            log_statements.append(f'photo diode used successfully ')
            events[:, 0] = pd_events
    else: 
        # if the pd channel didn't work, add 5 samples as a standard delay
        events[:,0] = events[:,0]+5
        log_statements.append(f'photo diode did not record anything. using parallel port trigger ')

    return raw,events,log_statements, pd_onsets, pd_offsets
def prepMarkers():
    subs   = ['S01']
    block_conditions = {
        'Watch_Still': ['0022_Left', '0022_Right', '0067_Left', '0067_Right', '0122_Left', '0122_Right', 
                        '0157_Left', '0157_Right', '0202_Left', '0202_Right', '0247_Left', '0247_Right',
                        '0292_Left', '0292_Right', '0337_Left', '0337_Right']}
    trial_data = []
    blocks = list(block_conditions.keys())
    for block in blocks:
        conditions = block_conditions[block]
        for condition in conditions:
            block_data = {'Block': block, 'Condition': condition}
            trial_data.append(block_data)

    df = pd.DataFrame(trial_data)
    print(df)
    df['Code'] = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
    ids = df['Code']
    conds = df['Block']
    orients = df['Condition']
    event_id = {}
    for i, id in enumerate(ids):
        event_id[id] = f'{subs[0]}/{conds[i]}/{orients[i]}'
    #keys = [f'{sub}/{cond}/{orient}/{id}' for sub in subs for id in ids for cond in conds for orient in orients]
    return event_id

In [14]:
#*****************************#
### PARAMETERS ###
#*****************************#

l_freq                      = 0.1
h_freq                      = 100
notch                       = 60
notch_max                   = 240
pre_stim_time               = -.2
post_stim_time              = 3.2
std_deviations_above_below  = 4
output_resolution           = 200
trigger_channel             = 'UPPT001'


dsets = []
for i in range(1,2):
    print(i)
    ds = f'00{i}'
    dsets.append(raw_dir + glob.glob('*_' + ds +'.ds',root_dir=raw_dir)[0]) 
#dsets.append(raw_dir + glob.glob('*_' + '006' +'.ds',root_dir=raw_dir)[0]) 
print(dsets)


event_ids = prepMarkers()
event_id = {v:k for k,v in event_ids.items()}

1
['/misc/data12/sjapee/Sebastian-OrientationImagery/Data/20240629/test360/MEG_OrientationImagery_20240629_001.ds']
          Block   Condition
0   Watch_Still   0022_Left
1   Watch_Still  0022_Right
2   Watch_Still   0067_Left
3   Watch_Still  0067_Right
4   Watch_Still   0122_Left
5   Watch_Still  0122_Right
6   Watch_Still   0157_Left
7   Watch_Still  0157_Right
8   Watch_Still   0202_Left
9   Watch_Still  0202_Right
10  Watch_Still   0247_Left
11  Watch_Still  0247_Right
12  Watch_Still   0292_Left
13  Watch_Still  0292_Right
14  Watch_Still   0337_Left
15  Watch_Still  0337_Right


In [15]:
pd_time = []

for i in range(len(dsets)):
    raw = mne.io.read_raw_ctf(dsets[i],system_clock='ignore',preload=True)
    events = mne.find_events(raw, stim_channel='UPPT001',min_duration = 0.002)
    raw, events, log_statements, pd_onsets, pd_offsets = defineOnsets2(raw, events)
    pd_onsets_np = np.array(pd_onsets)
    pd_offsets_np = np.array(pd_offsets)
    offsets_minus_onsets_np = pd_offsets_np - pd_onsets_np
    pd_time.extend(offsets_minus_onsets_np)

ds directory : /misc/data12/sjapee/Sebastian-OrientationImagery/Data/20240629/test360/MEG_OrientationImagery_20240629_001.ds
    res4 data read.
    hc data read.
    Separate EEG position data file not present.
    Quaternion matching (desired vs. transformed):
       0.00   80.00    0.00 mm <->    0.00   80.00    0.00 mm (orig :  -56.57   56.57 -210.00 mm) diff =    0.000 mm
       0.00  -80.00    0.00 mm <->    0.00  -80.00    0.00 mm (orig :   56.57  -56.57 -210.00 mm) diff =    0.000 mm
      80.00    0.00    0.00 mm <->   80.00   -0.00    0.00 mm (orig :   56.57   56.57 -210.00 mm) diff =    0.000 mm
    Coordinate transformations established.
    Polhemus data for 3 HPI coils added
    Device coordinate locations for 3 HPI coils added
    Measurement info composed.
Finding samples for /misc/data12/sjapee/Sebastian-OrientationImagery/Data/20240629/test360/MEG_OrientationImagery_20240629_001.ds/MEG_OrientationImagery_20240629_001.meg4: 
    System clock channel is available, but i

In [16]:
# Get unique values and their counts
unique_values, counts = np.unique(pd_time, return_counts=True)

# Find indices for each unique value
indices = [np.where(pd_time == value)[0] for value in unique_values]

# Print unique values, counts, and indices
for value, count, idx in zip(unique_values, counts, indices):
    print(f"Value: {value}, Count: {count}, Indices: {idx}")

Value: 139, Count: 3, Indices: [0 1 2]
Value: 3718, Count: 52, Indices: [  3   6   7   9  13  16  19  20  23  26  29  30  32  33  36  39  42  43
  46  49  55  56  59  62  63  65  66  69  72  75  76  78  79  82  85  86
  92  95  96  98  99 102 103 107 108 110 111 114 118 122 125 128]
Value: 3719, Count: 58, Indices: [  4   5   8  11  12  14  15  18  22  25  27  28  34  35  37  40  41  44
  47  48  50  51  53  54  57  58  60  61  64  68  70  71  73  77  80  81
  83  84  87  88  90  91  93  94 100 101 106 109 112 115 116 117 119 120
 123 124 126 127]
Value: 3738, Count: 1, Indices: [104]
Value: 3739, Count: 1, Indices: [21]
Value: 4518, Count: 3, Indices: [ 52  89 121]
Value: 4519, Count: 10, Indices: [ 17  24  31  38  45  67  74  97 105 113]
Value: 4598, Count: 1, Indices: [10]
